In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
import logging
logger = logging.getLogger("mlflow")
logger.setLevel(logging.DEBUG)

In [ ]:
# import mlflow
# mlflow.tracing.enable()

# from mlflow.tracing.export.mlflow import MlflowSpanExporter
# from mlflow.tracing.processor.mlflow import MlflowSpanProcessor
# from mlflow.tracing.processor.otel import OtelSpanProcessor
# from mlflow.tracing.utils.otlp import get_otlp_exporter

from opentelemetry.sdk.trace import TracerProvider
# exporter = MlflowSpanExporter()
# processor = MlflowSpanProcessor(exporter)
# exporter2 = get_otlp_exporter()
# processor2 = OtelSpanProcessor(exporter2)

tracer_provider = TracerProvider()
# tracer_provider.add_span_processor(processor2)

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter

# 1. Set up the provider and exporter (OTLP exporter uses env vars for config)
otlp_exporter = OTLPSpanExporter()
processor = SimpleSpanProcessor(otlp_exporter)
tracer_provider.add_span_processor(processor)

# 2. Register the provider as global
trace.set_tracer_provider(tracer_provider)

In [ ]:
# mlflow.set_experiment("SK Tracing")

In [ ]:
import os

assert os.getenv("SEMANTICKERNEL_EXPERIMENTAL_GENAI_ENABLE_OTEL_DIAGNOSTICS_SENSITIVE") != ""

In [ ]:
import os
from openai import AsyncAzureOpenAI
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential, "https://cognitiveservices.azure.com/.default"
)


client = AsyncAzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        azure_ad_token_provider=token_provider,
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    )
service = AzureChatCompletion(
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        async_client=client,
    )

In [ ]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.agents.chat_completion.chat_completion_agent import ChatHistoryAgentThread

thread = ChatHistoryAgentThread()

agent = ChatCompletionAgent(
    id="chat-completion-agent",
    name="ChatCompletionAgent",
    service=service,
    instructions="""You are a helpful assistant. You tell jokes""",
)

In [ ]:
import mlflow

# 3. Start an MLflow run (optional if you want a top-level root span)
with mlflow.start_run(run_name="SK Traced Run"):
    async for r in agent.invoke(messages="A joke about developers", thread=thread):
        print(r)

In [ ]:
finished = memory_exporter.get_finished_spans()
print(f"Captured {len(finished)} spans: {[span.name for span in finished]}")

In [ ]:
import mlflow
# High-level API returns a pandas DataFrame
traces_df = mlflow.search_traces(
    filter_string="",
    order_by=["timestamp DESC"],
    max_results=50
)
print(traces_df.head())

In [ ]:
exporter2.force_flush()